In [79]:
import pandas as pd
import numpy as np
from pulp import *

In [80]:
nr_pos = np.random.randint(low=4, high=5)
choices_per_pos = np.random.randint(low=5, high=7, size=nr_pos)

In [81]:
muls=list()
dims=list()
for mul in range(0,nr_pos):
    for dim in range(0,choices_per_pos[mul]):
        muls.append('mul_'+str(mul))
        dims.append('dim_'+str(dim))
df = pd.DataFrame(data = {'pos': muls,
                         'choices': dims}
                  , columns=['pos','choices'])
df.loc[:, 'values'] = np.random.randint(low=2, high=8, size=len(df))

In [82]:
df = df.set_index(['pos','choices'])

In [83]:
pos_choice = pulp.LpVariable.dicts("pos_choice",
                                     ((pos, choice) for pos, choice in df.index),
                                     cat='Binary')

In [84]:
model = pulp.LpProblem("dummy", pulp.LpMaximize)

model += pulp.lpSum(
    [pos_choice[pos, choice] * df.loc[(pos, choice), 'values'] for pos, choice in df.index]
)

In [85]:
for mull in df.index.levels[0]:
    model += sum([pos_choice[comb] for comb in df.index if comb[0]==mull]) <= 1
    model += sum([pos_choice[comb] for comb in df.index if comb[0]==mull]) >= 1

model += pulp.lpSum(
    [pos_choice[pos, choice] * df.loc[(pos, choice), 'values'] for pos, choice in df.index]
)<=13

In [86]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [87]:
output = []
for pos, choice in df.index:
    if pos_choice[(pos, choice)].varValue==1:
        var_output = {
            'pos': pos,
            'choices': choice,
            'choice_made': pos_choice[(pos, choice)].varValue
        }
        output.append(var_output)
output_df = pd.DataFrame.from_records(output)

In [88]:
output_df

,choice_made,choices,pos
0,1.0,dim_3,mul_0
1,1.0,dim_2,mul_1
2,1.0,dim_3,mul_2
3,1.0,dim_4,mul_3


In [89]:
df.reset_index().merge(output_df)

,pos,choices,values,choice_made
0,mul_0,dim_3,3,1.0
1,mul_1,dim_2,3,1.0
2,mul_2,dim_3,5,1.0
3,mul_3,dim_4,2,1.0
